In [0]:
%%sh
aws s3 ls s3://b2c-prod-dca-store-estimates/store_estv2/APP_ESTIMATES_FINAL/version=2.0.0/range_type=DAY/date=2020-06-20/  --recursive
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est.v1/fact/granularity=daily/date=2020-06-20 --recursive

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-category-pre-aggr.v3/fact/granularity=weekly/date=2019-01-12/device_code=ios-phone/
 
# store.download-attribution-pre-aggr.v3/


In [0]:

spark.read.parquet("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/granularity=weekly/date=2019-01-12/device_code=ios-phone/").show()

In [0]:

import datetime

start = "2020-03-29"
end = "2020-05-02"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
print sar_list
for d in sar_list:
    daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where("granularity='daily' and date in ('%s')" % ("','".join(sar_list[0][1])))
    daily_download_attr.createOrReplaceTempView("daily_pre_load_data")
    
    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly' and date in ('%s')" % (sar_list[0][0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    

    spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

    spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()



In [0]:

import datetime

start = "2020-02-01"
end = "2020-04-01"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
# sar_list=list()
for days in xrange(date_range.days):
    dates.append(str(real_date1 + datetime.timedelta(days)))
#     if (real_date1 + datetime.timedelta(days)).weekday() == 5:
#         temp=list()
#         while dates:
#             temp.append(str(dates.pop()))
#         sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
# print sar_list

print dates
print dates[-1]
# sql_where = "granularity='daily' and date in ('%s')" % ("','".join(dates))
sql_where = "granularity='daily' and date between '2019-03-01' and '2019-03-31'"

print sql_where
daily_download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where(sql_where)
daily_download_attr.createOrReplaceTempView("daily_pre_load_data")

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='monthly' and date='2019-03-31' ")
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    
spark.sql("select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified except all select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code").show()

spark.sql("select app_id, country_code, device_code, sum(est_organic_download) as est_organic_download, sum(est_paid_download) as est_paid_download from daily_pre_load_data group by app_id, country_code, device_code except all select app_id, country_code, device_code, est_organic_download, est_paid_download from download_attribution_weekly_unified").show()


spark.sql("select * from download_attribution_weekly_unified").show()

In [0]:

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='monthly'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")

test_date_list = spark.sql("select distinct date from download_attribution_weekly_unified").collect()

l = [str(d[0]) for d in test_date_list]
l.sort(reverse=False)
for x in l:
    print x

In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/granularity=monthly/
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2020-03-22/

In [0]:

import datetime

start = "2016-08-28"
end = "2020-05-31"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days))))

print sar_list

In [0]:

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")

test_date_list = spark.sql("select distinct date from download_attribution_weekly_unified").collect()

l = [str(d[0]) for d in test_date_list]
l.sort(reverse=False)
print l

In [0]:

daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-pre-aggr.v3/fact/").where("granularity='weekly'" )
daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")

spark.sql("select * from download_attribution_weekly_unified where date = '2020-04-09' and granularity='weekly'  ").show()

In [0]:

import datetime
import calendar

start = "2010-07-04"
end = "2010-08-04"
# end = "2012-05-01"
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()

month_day=list()
for days in xrange(date_range.days):
    month_day.append(real_date1 + datetime.timedelta(days))


test_list= sorted(list(set([ d.strftime("%Y-%m-%d")[:7] for d in month_day ])))

print test_list

# for m in test_list:
#     print calendar.monthrange(int(m[:4]),int(m[5:7]))
#     print "-".join([str(m[:4]), str(calendar.monthrange(int(m[:4]),int(m[-2:]))[0]), str(calendar.monthrange(int(m[:4]),int(m[-2:]))[1])])


def last_day_of_month(date):
    next_month = date.replace(day=28) + datetime.timedelta(days=4)  # this will never fail
    return next_month - datetime.timedelta(days=next_month.day)
print last_day_of_month(start)

In [0]:
%%sh
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/granularity=daily/date=2017
aws s3 ls s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-load.v3/fact/granularity=daily/date=2017
 


In [0]:

download_attr = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.download-attribution-load.v3/fact/").where("granularity='daily' and date between '2017-04-01' and '2017-06-30'").cache()
download_attr.createOrReplaceTempView("download_attr")


est = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/store.app-est-load.v3/fact/").where("granularity='daily' and date between '2017-04-01' and '2017-06-30'").cache()
est.createOrReplaceTempView("est")

spark.sql("select date, count(date) as date_count from download_attr where app_id= 20600002647354  and country_code='US' group by date ").show(10000)
spark.sql("select date, count(date) as date_count from est where app_id= 20600002647354 and country_code='US' group by date ").show(10000)

# spark.sql("select date,count(date) ( select date from download_attr ) as prod group by date ").show(1000)
# spark.sql("select date,count(date) (select date from est ) as prod group by date").show(1000)


In [0]:
%%sh
date

In [0]:

import datetime
import psycopg2
import datetime
spark.sparkContext.addPyFile("/home/hadoop/bdp/application/libs/python/dependencies.zip")
import aaplproxy
from aadatapipelinecore.core.urn import Urn
from aaplproxy.da.local_sqlrunner import LocalSqlRunner
from aadatapipelinecore.core.utils.module import application_settings
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import functions as F

start = '2020-02-23'
end = '2020-03-01'
granularity = 'weekly'


granularity_table_dict = {'weekly': ['store.app-est.v3', 'store_est_t_w_fact_v1', ''],
                         'monthly': ['store.app-est.v3', 'store_est_t_m_fact_v1'],
                         'quarterly' : ['store.app-est-pre-aggr.v3', 'store_est_t_q_fact_v1'],
                         'yearly' : ['store.app-est-pre-aggr.v3', 'store_est_t_y_fact_v1']
                        }

download_attribution_bucket='store.download-attribution-pre-aggr.v3'

real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range = real_date2 - real_date1
dates = list()
sar_list=list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
    if (real_date1 + datetime.timedelta(days)).weekday() == 5:
        temp=list()
        while dates:
            temp.append(str(dates.pop()))
        sar_list.append((str(real_date1 + datetime.timedelta(days)), temp))
print sar_list



def citus_row(date, granularity):
    def get_data_in_citus(date):
        print 'get data in citus ', date
        citus_dsn_ = (
            "dbname='{db}' user='{user}' password='{password}' "
            "host='{host}' port='{port}'".format(
                db="aa_store_db",
                user="citus_bdp_prod_app_int_qa",
                host="10.2.6.141",
                password="wZw8cfBuuklIskVG",
                port=5432
            )
        )
        sql = '''SELECT  
                    cast(sum(est_free_app_download) as bigint) as est_free_app_download, 
                    cast(sum(est_paid_app_download) as bigint) as est_paid_app_download, 
                    cast(sum(est_revenue) as bigint) as est_revenue, 
                    cast(sum(est_organic_download) as bigint) as est_organic_download ,  
                    cast(sum(est_paid_download) as bigint) as est_paid_download
                FROM store.{}
                WHERE date in ('{}') '''.format(granularity_table_dict[granularity][1],date)
        db_data = query(citus_dsn_, sql)
        return db_data
    def query(dsn, sql):
        with psycopg2.connect(dsn) as conn:
            conn.autocommit = True
            with conn.cursor() as cur:
                cur.execute(sql)
                result = cur.fetchall()
                conn.commit()
        return result
    result = get_data_in_citus(date)
    return [Row(sum_est_free_app_download=r[0], sum_est_paid_app_download=r[1], sum_est_revenue=r[2], sum_est_organic_download=r[3], sum_est_paid_download=r[4] ) for r in result]


for d in sar_list:
    print d[0]

    daily_download_attr_preload_unified = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/{}/fact/".format(download_attribution_bucket)).where("granularity='%s' and date in ('%s')" % (granularity, d[0]))
    daily_download_attr_preload_unified.createOrReplaceTempView("download_attribution_weekly_unified")
    
    est_weekly_df = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-store-paid/unified/" \
                    "{}/fact/".format(granularity_table_dict[granularity][0])).where("granularity='{}' and date in ('{}')".format(granularity,d[0] ))
    est_weekly_df.createOrReplaceTempView("est_weekly_df")


    citus_result = citus_row(d[0], 'weekly')

    schema = StructType([
    StructField("sum_est_free_app_download", LongType(), True),
    StructField("sum_est_paid_app_download", LongType(), True),
    StructField("sum_est_revenue", LongType(), True),
    StructField("sum_est_organic_download", LongType(), True),
    StructField("sum_est_paid_download", LongType(), True)])

    df_3 = spark.createDataFrame(citus_result, schema)
    df_3.createOrReplaceTempView("citus_data")

    
    # compare est paid download by DB and download attribution
    spark.sql("select sum(est_paid_app_download) as sum_est_paid_download from download_attribution_weekly_unified except all select sum_est_paid_download from citus_data").show()
    spark.sql("select sum_est_paid_download from citus_data except all select sum(est_paid_app_download) as sum_est_paid_download from download_attribution_weekly_unified").show()

    # compare est weekly data with DB metric sum(free_app_download), sum(paid_app_download), sum(revenue) 
    spark.sql("select sum(free_app_download), sum(paid_app_download), sum(revenue) from est_weekly_df except all select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data ").show()
    spark.sql("select sum_est_free_app_download, sum_est_paid_app_download, sum_est_revenue from citus_data except all select sum(free_app_download), sum(paid_app_download), sum(revenue) from est_weekly_df ").show()


    # compare organic + paid = free_app + paid_app in DB
    list_result =  spark.sql("select sum_est_free_app_download + sum_est_paid_app_download,  sum_est_organic_download + sum_est_paid_download from citus_data ").collect()
    if list_result[0][0] != list_result[0][1]:
        print 'compare compare organic + paid = free_app + paid_app failed!! , organic + paid= {}, free_app + paid_app = {}'.format(list_result[0][0], list_result[0][1])
    else:
        print 'pass!'

In [0]:


spark.sql("select * from citus_data").show()
# list_result =  spark.sql("select sum_est_free_app_download + sum_est_paid_app_download,  sum_est_organic_download + sum_est_paid_download from citus_data ").collect()

# print list_result[0][0]
# print list_result[0][1]

In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;
select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue),  sum(est_organic_download),  sum(est_paid_download) from store.store_est_fact_v1 where date between '2020-02-23' and '2020-02-29' and granularity='daily' limit 3 ;
select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue),  sum(est_organic_download),  sum(est_paid_download) from store.store_est_t_w_fact_v1 where date between  '2020-02-23' and '2020-02-29' and granularity='weekly' limit 3 ;
SELECT  
                    cast(sum(est_free_app_download) as bigint	) as est_free_app_download, 
                    cast(sum(est_paid_app_download) as bigint	) as est_paid_app_download, 
                    cast(sum(est_revenue) as bigint	) as est_revenue, 
                    cast(sum(est_organic_download) as bigint	) as est_organic_download ,  
                    cast(sum(est_paid_download) as bigint	) as est_paid_download
                FROM store.store_est_t_w_fact_v1
                WHERE date in ('2020-02-29')
EOF

# select sum(est_free_app_download), sum(est_paid_app_download), sum(est_revenue),  sum(est_organic_download),  sum(est_paid_download) from store.store_est_t_w_fact_v1 where date = '2017-07-15' and granularity='weekly' limit 3 ;
# select distinct date from store.store_est_t_w_fact_v1 where date between '2017-06-25' and '2017-07-15' and granularity='weekly' limit 10;


In [0]:
%%sh
PGPASSWORD='wZw8cfBuuklIskVG' psql -h 10.2.6.141  -U citus_bdp_prod_app_int_qa -d aa_store_db -p 5432 << EOF 
set search_path=store;

select * from store.store_est_category_t_w_fact_v1 limit 3 ;
select * from store.store_est_category_t_m_fact_v1 where date='2019-01-12' limit 3 ;
select * from store.store_est_category_t_q_fact_v1 limit 3 ;
select * from store.store_est_category_t_y_fact_v1 limit 3 ;

EOF

In [0]:

spark.sql("select * from est_weekly_df where app_id=20600004908242 and country_code='WW'").show()
spark.sql("select * from download_attribution_weekly_unified where app_id=20600004908242 and country_code='WW'").show()